In [1]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

In [2]:
# Executer ci dessous ci besoin pour récupérer les données
# !extract-sample-data
# Et pour sauver une version markdown des notebooks, utiliser
# jupyter nbconvert verif_haut_niveau.ipynb --to markdown --output-dir=exports/

In [ ]:
src = DATA_FOLDER / "saint_denis_reference_data.gpkg"
layers = fiona.listlayers(src)

for layer in layers:
    gdf_sd = gpd.read_file(src, layer=layer)
    print(f"* Couche: {layer} avec {len(gdf_sd)} éléments.")

In [4]:
zones = gpd.read_file(src, layer="bdtopo_education")
ecoles = gpd.read_file(src, layer="annuaire_education")
ecoles_seules = gpd.read_file(src, layer="annuaire_education_sans_zone") 
potentiel = gpd.read_file(src, layer="potentielsolaire_toitures")
batiments_ecoles = gpd.read_file(src, layer="bdtopo_batiment")
stdenis = gpd.read_file(src, layer="perimetre_st_denis")

In [ ]:
fig1, ax1 = plt.subplots(figsize=(15, 15))
zones.plot(ax=ax1, color='black', alpha=0.5, label='Zones')
stdenis.plot(ax=ax1, color='black', alpha=0.5, label='Zones')
ecoles.plot(ax=ax1, color='green', alpha=0.5, label='Ecoles avec zone')
ecoles_seules.plot(ax=ax1, color='red', alpha=0.5, label='Ecole seules sans zone')
batiments_ecoles.plot(ax=ax1, facecolor='yellow', edgecolor='red', linewidth=0.5, alpha=0.8, label='Saint-Denis')
potentiel.plot(ax=ax1, facecolor='orange', edgecolor='red', linewidth=0.5, alpha=0.8, label='Saint-Denis')
cx.add_basemap(ax1, crs=zones.crs, zoom=14)
ax1.set_title(f"Bâtiments à Saint-Denis\n{len(potentiel)} bâtiments", pad=20)
ax1.legend()
plt.show()

# Premiere estimation

France: 1000 kWh/m2/an  à 1900 kWh/m2/an

In [6]:
gdf = potentiel

gdb = batiments_ecoles

In [7]:
batiments_calcules = gdf[list(gdf.columns)[:18]+["geometry"]]
batiments_calcules = batiments_calcules.rename(columns={"id": "cleabs"})
batiments_reference = gdb
batiments_reference = batiments_reference.rename(columns={"cleabs_left__bat": "cleabs"})
communs = list(set(batiments_calcules.cleabs) & set(batiments_reference.cleabs))
batiments_reference = batiments_reference[batiments_reference.cleabs.isin(communs)]
batiments_calcules = batiments_calcules[batiments_calcules.cleabs.isin(communs)]


# On calcule les surfaces
batiments_calcules = batiments_calcules.to_crs(epsg=6933)
batiments_calcules["surface_calculee"] = batiments_calcules.area
batiments_calcules = batiments_calcules.to_crs(epsg=4326)

batiments_reference = batiments_reference.to_crs(epsg=6933)
batiments_reference["surface_calculee"] = batiments_reference.area
batiments_reference = batiments_reference.to_crs(epsg=4326)

In [8]:
batiments_calcules["calcul_naif"] = 1100*batiments_calcules["surface_calculee"]

In [ ]:
batiments_calcules = batiments_calcules[["surface_calculee", "surf_util", "production", "calcul_naif","moyenne2", "indic2","forme",  "gisement"	,"eq_pano",	"eq_surf"	,"cleabs", "geometry"]]
test = batiments_calcules[batiments_calcules.surf_util > 0]
test["ratio"] = test["production"] / test["calcul_naif"]
ratio = test["ratio"].mean()
batiments_calcules["calcul_naif"] = 1100*batiments_calcules["surface_calculee"]
batiments_calcules["calcul_naif_maj"] = 1100*batiments_calcules["surface_calculee"]*ratio
batiments_calcules["ratio_surface"] = batiments_calcules["surf_util"]/batiments_calcules["surface_calculee"]
batiments_calcules

In [ ]:
batiments_calcules.plot.scatter(x="surface_calculee",y="production",c="indic2",colormap='viridis',\
    title='Relation between production et surface calculée\nloglog scale', \
    logx=True, logy=True,
    figsize = (10,5))

In [ ]:
batiments_calcules[batiments_calcules.indic2 > 0].plot.scatter(x="surface_calculee",y="surf_util",c="indic2",colormap='viridis', \
    title='Relation between surface calculée et surface utile\nloglog scale', \
    logx=True, logy=True,
    figsize = (10,5))

In [ ]:
batiments_calcules[batiments_calcules.indic2 > 0].plot.scatter( \
    x="production",y="calcul_naif_maj",c="indic2",colormap='viridis', \
    title='Relation between naive estimation and actual production\nloglog scale', \
    logx=True, logy=True,
    figsize = (10,5))

In [ ]:
batiments_calcules.forme.unique()

In [ ]:
batiments_calcules.groupby('forme')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


groups = batiments_calcules.groupby('forme')

# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    if name == "plat":
        pass
    else:
        ax.plot(group.production, group.ratio_surface, marker='o', linestyle='', label=name)
ax.legend()
plt.title("Revue des ratios surfaceutile / surfacetotale en fonction des types de toits\n")
plt.show()

In [ ]:
groups = batiments_calcules.groupby('forme')

# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    if name == "plat":
        ax.plot(group.production, group.ratio_surface, marker='o', linestyle='', label=name)
ax.legend()
plt.title("Revue des ratios surfaceutile / surfacetotale pour les toits plats\n")
plt.show()

# Ajout de la layer potentiel solaire 

PV_Sol_Polygon.gpkg
* Couche: PV_Sol avec 2016422 éléments.
* Couche: layer_styles avec 1 éléments.

In [17]:
mask_stdenis = stdenis.to_crs(2154)

In [18]:
src = DATA_FOLDER / "ENR_1-0_POT-SOL-SOL_GPKG_LAMB93_FXX_2024-04-01/1_DONNEES_LIVRAISON/PV_Sol_Polygon.gpkg"
layers = fiona.listlayers(src)


In [ ]:
enrpot = gpd.read_file(src, layer="PV_Sol", mask = mask_stdenis)
enrpot

# Representing the area for StDenis

In [ ]:
enrpot.clip(mask_stdenis).plot()